In [12]:
!pip install streamlit pyngrok PyJWT bcrypt --quiet

In [40]:
%%writefile app.py
import streamlit as st
import sqlite3
import re
import jwt
import datetime
import bcrypt

SECRET_KEY = "policynav_secret_key"

# ══════════════════════════════════════════════════════════════════
#  CSS  ─ editorial minimal, warm cream, Playfair + DM Sans
# ══════════════════════════════════════════════════════════════════
def load_css():
    st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Playfair+Display:ital,wght@0,700;1,400&family=DM+Sans:wght@300;400;500&display=swap');

    .stApp { background-color: #f4f1ec; font-family: 'DM Sans', sans-serif; }
    .block-container { padding-top: 0 !important; padding-bottom: 0 !important; max-width: 100% !important; }
    #MainMenu, footer, header, [data-testid="stToolbar"], [data-testid="stDecoration"] { visibility: hidden; }

    /* Logo */
    .logo {
        position: fixed; top: 24px; left: 28px;
        display: flex; align-items: center; gap: 9px; z-index: 999;
    }
    .logo-icon {
        width: 40px; height: 40px; background: #1a1a1a; border-radius: 10px;
        display: flex; align-items: center; justify-content: center;
        font-size: 17px; color: white;
    }
    .logo-name { font-size: 14px; font-weight: 500; color: #1a1a1a; letter-spacing: -0.2px; }
    .logo-name span { font-weight: 300; color: #9a948b; }

    /* Hero */
    .hero {
        display: flex; flex-direction: column; align-items: center;
        justify-content: center; min-height: 44vh;
        text-align: center; padding: 60px 24px 24px;
    }
    .hero-eye {
        font-size: 10.5px; font-weight: 500; letter-spacing: 3.5px;
        color: #9a948b; text-transform: uppercase; margin-bottom: 22px;
    }
    .hero-h1 {
        font-family: 'Playfair Display', serif; font-weight: 700;
        font-size: clamp(50px, 6.5vw, 78px); color: #1a1a1a;
        line-height: 1.06; margin-bottom: 18px;
    }
    .hero-h1 em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .hero-sub {
        font-size: 15px; color: #9a948b; font-weight: 300;
        margin-bottom: 8px; letter-spacing: 0.1px;
    }

    /* Form header */
    .fh { text-align: center; padding: 80px 0 0; }
    .fh-eye {
        font-size: 10.5px; letter-spacing: 3.5px; color: #9a948b;
        text-transform: uppercase; margin-bottom: 10px;
    }
    .fh-title {
        font-family: 'Playfair Display', serif; font-size: 38px;
        font-weight: 700; color: #1a1a1a; margin-bottom: 8px;
    }
    .fh-title em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .fh-sub { font-size: 14px; color: #9a948b; font-weight: 300; margin-bottom: 0; }
    .fh-rule { width: 36px; height: 1px; background: #d4cfc8; margin: 28px auto 32px; }

    /* Inputs */
    .stTextInput > label {
        font-size: 11px !important; font-weight: 500 !important;
        letter-spacing: 1.8px !important; text-transform: uppercase !important;
        color: #7c7c7c !important; margin-bottom: 5px !important;
    }
    .stTextInput > div > div > input {
        background: #ffffff !important; border: 1px solid #e0dbd3 !important;
        border-radius: 10px !important; padding: 14px 16px !important;
        font-family: 'DM Sans', sans-serif !important; font-size: 15px !important;
        color: #1a1a1a !important; box-shadow: 0 1px 3px rgba(0,0,0,0.04) !important;
        transition: border-color 0.18s !important;
    }
    .stTextInput > div > div > input:focus {
        border-color: #1a1a1a !important;
        box-shadow: 0 0 0 3px rgba(26,26,26,0.07) !important;
    }
    input { color: #1a1a1a !important; -webkit-text-fill-color: #1a1a1a !important; }
    input::placeholder { color: #b8b2aa !important; font-weight: 300 !important; }

    .stSelectbox > label {
        font-size: 11px !important; font-weight: 500 !important;
        letter-spacing: 1.8px !important; text-transform: uppercase !important;
        color: #7c7c7c !important;
    }
    div[data-baseweb="select"] > div {
        background: #ffffff !important; border: 1px solid #e0dbd3 !important;
        border-radius: 10px !important; color: #1a1a1a !important;
    }

    /* Primary button */
    .stButton > button {
        font-family: 'DM Sans', sans-serif !important; font-size: 14px !important;
        font-weight: 500 !important; letter-spacing: 0.2px !important;
        background-color: #1a1a1a !important; color: #ffffff !important;
        border: none !important; border-radius: 100px !important;
        padding: 13px 28px !important; width: 100% !important;
        transition: all 0.18s ease !important;
    }
    .stButton > button:hover {
        background-color: #2d2d2d !important;
        transform: translateY(-1px) !important;
        box-shadow: 0 5px 18px rgba(0,0,0,0.16) !important;
    }
    .stButton > button:active { transform: translateY(0) !important; }

    /* Ghost / back button */
    .ghost .stButton > button {
        background-color: transparent !important; color: #7c7c7c !important;
        border: 1px solid #d4cfc8 !important; font-size: 13px !important;
    }
    .ghost .stButton > button:hover {
        background-color: #ede9e2 !important; color: #1a1a1a !important;
        box-shadow: none !important;
    }

    /* Alerts */
    .stSuccess > div {
        background: #f2faf5 !important; border: 1px solid #a7d9b8 !important;
        border-radius: 10px !important; color: #145a32 !important;
        font-family: 'DM Sans', sans-serif !important; font-size: 14px !important;
    }
    .stError > div {
        background: #fff6f6 !important; border: 1px solid #f5bcbc !important;
        border-radius: 10px !important; color: #7b1a1a !important;
        font-family: 'DM Sans', sans-serif !important; font-size: 14px !important;
    }
    .stInfo > div {
        background: #f5f3ff !important; border: 1px solid #c4b9f5 !important;
        border-radius: 10px !important;
        font-family: 'DM Sans', sans-serif !important; font-size: 14px !important;
    }

    /* Dashboard welcome card */
    .dash-card {
        background: #ffffff; border-radius: 18px; padding: 44px 40px;
        box-shadow: 0 2px 24px rgba(0,0,0,0.06); text-align: center; margin-bottom: 24px;
    }
    .dash-card h2 {
        font-family: 'Playfair Display', serif; font-size: 30px;
        color: #1a1a1a; margin-bottom: 6px; line-height: 1.3;
    }
    .dash-card h2 em { font-style: italic; font-weight: 400; color: #7c7c7c; }
    .dash-card p { font-size: 13px; color: #9a948b; margin: 0; }

    /* Spacers */
    .sp8  { height: 8px; }
    .sp16 { height: 16px; }
    .sp24 { height: 24px; }
    </style>
    """, unsafe_allow_html=True)


# ══════════════════════════════════════════════════════════════════
#  DATABASE
# ══════════════════════════════════════════════════════════════════
def init_db():
    conn = sqlite3.connect("users.db", check_same_thread=False)
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS users (
        id         INTEGER PRIMARY KEY AUTOINCREMENT,
        username   TEXT NOT NULL,
        email      TEXT UNIQUE NOT NULL,
        password   BLOB NOT NULL,
        security_q TEXT NOT NULL,
        security_a BLOB NOT NULL
    )""")
    conn.commit(); conn.close()

def validate_email(e): return re.match(r"[^@]+@[^@]+\.[a-zA-Z]{2,}", e)
def validate_password(p): return p.isalnum() and len(p) >= 8
def hash_text(t): return bcrypt.hashpw(t.encode(), bcrypt.gensalt())
def verify_text(t, h): return bcrypt.checkpw(t.encode(), h)

def create_token(email, username):
    return jwt.encode({
        "email": email, "username": username,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(minutes=30)
    }, SECRET_KEY, algorithm="HS256")

def verify_token(token):
    try: return jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
    except: return None


# ── Session defaults ──────────────────────────────────────────────
for k, v in [("page","home"),("reset_email",None),
             ("question_shown",False),("answer_verified",False)]:
    if k not in st.session_state:
        st.session_state[k] = v


# ══════════════════════════════════════════════════════════════════
#  HELPERS
# ══════════════════════════════════════════════════════════════════
def logo():
    st.markdown("""
    <div class="logo">
      <div class="logo-icon">◈</div>
      <div class="logo-name">Policy<span>Nav</span></div>
    </div>""", unsafe_allow_html=True)

def sp(cls="sp8"):
    st.markdown(f"<div class='{cls}'></div>", unsafe_allow_html=True)

def ghost_back(label, dest, key):
    st.markdown("<div class='ghost'>", unsafe_allow_html=True)
    if st.button(label, key=key):
        st.session_state.page = dest; st.rerun()
    st.markdown("</div>", unsafe_allow_html=True)


# ══════════════════════════════════════════════════════════════════
#  MAIN
# ══════════════════════════════════════════════════════════════════
def main():
    st.set_page_config(page_title="PolicyNav", page_icon="◈", layout="centered")
    load_css(); init_db(); logo()

    # ── HOME ──────────────────────────────────────────────────────
    if st.session_state.page == "home":
        st.markdown("""
        <div class="hero">
          <div class="hero-eye">Secure Authentication</div>
          <div class="hero-h1">Navigate with<br><em>confidence.</em></div>
          <div class="hero-sub">Simple, secure access to your policy dashboard.</div>
        </div>""", unsafe_allow_html=True)

        _, col, _ = st.columns([1.2, 1, 1.2])
        with col:
            if st.button("Sign in",        key="h_li"): st.session_state.page="login";  st.rerun()
            sp()
            if st.button("Create account", key="h_su"): st.session_state.page="signup"; st.rerun()
            sp()
            if st.button("Forgot password",key="h_fp"): st.session_state.page="forgot"; st.rerun()

    # ── LOGIN ─────────────────────────────────────────────────────
    elif st.session_state.page == "login":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Welcome back</div>
          <div class="fh-title">Sign <em>in.</em></div>
          <div class="fh-sub">Enter your credentials to continue.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)

        _, col, _ = st.columns([1, 1.6, 1])
        with col:
            email = st.text_input("Email",    placeholder="you@example.com", key="li_em")
            pw    = st.text_input("Password", placeholder="••••••••",
                                  type="password", key="li_pw")
            sp("sp16")
            if st.button("Sign in →", key="li_btn"):
                if not email or not pw:
                    st.error("Please fill in all fields.")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    c.execute("SELECT username, password FROM users WHERE email=?", (email,))
                    user = c.fetchone(); conn.close()
                    if user and verify_text(pw, user[1]):
                        st.session_state.token    = create_token(email, user[0])
                        st.session_state.username = user[0]
                        st.session_state.page     = "dashboard"; st.rerun()
                    else:
                        st.error("Invalid email or password.")
            sp()
            ghost_back("← Back", "home", "li_back")

    # ── SIGN UP ───────────────────────────────────────────────────
    elif st.session_state.page == "signup":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Get started</div>
          <div class="fh-title">Create an <em>account.</em></div>
          <div class="fh-sub">Join PolicyNav in seconds.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)

        _, col, _ = st.columns([1, 1.6, 1])
        with col:
            username   = st.text_input("Username",         placeholder="Your name",               key="su_un")
            email      = st.text_input("Email",            placeholder="you@example.com",         key="su_em")
            pw         = st.text_input("Password",         placeholder="Min 8 alphanumeric chars",
                                        type="password",                                           key="su_pw")
            confirm    = st.text_input("Confirm Password", placeholder="Same as above",
                                        type="password",                                           key="su_cp")
            security_q = st.selectbox("Security Question", [
                "What is your pet name?",
                "What is your favourite colour?",
                "What is your favourite book?",
                "What is your favourite movie?",
                "Who is your favourite teacher?"
            ], key="su_sq")
            security_a = st.text_input("Security Answer",  placeholder="No spaces",               key="su_sa")
            sp("sp16")
            if st.button("Create account →", key="su_btn"):
                if not all([username, email, pw, confirm, security_a]):
                    st.error("All fields are required.")
                elif not validate_email(email):
                    st.error("Invalid email format.")
                elif not validate_password(pw):
                    st.error("Password must be alphanumeric and 8+ characters.")
                elif pw != confirm:
                    st.error("Passwords do not match.")
                elif " " in pw or " " in security_a:
                    st.error("No spaces allowed in password or security answer.")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    try:
                        c.execute(
                            "INSERT INTO users (username,email,password,security_q,security_a)"
                            " VALUES (?,?,?,?,?)",
                            (username, email, hash_text(pw),
                             security_q, hash_text(security_a.lower().strip())))
                        conn.commit()
                        st.success("Account created — please sign in.")
                    except sqlite3.IntegrityError:
                        st.error("Email already registered.")
                    conn.close()
            sp()
            ghost_back("← Back", "home", "su_back")

    # ── FORGOT PASSWORD ───────────────────────────────────────────
    elif st.session_state.page == "forgot":
        st.markdown("""
        <div class="fh">
          <div class="fh-eye">Account recovery</div>
          <div class="fh-title">Reset your <em>password.</em></div>
          <div class="fh-sub">Answer your security question to continue.</div>
          <div class="fh-rule"></div>
        </div>""", unsafe_allow_html=True)

        _, col, _ = st.columns([1, 1.6, 1])
        with col:
            email = st.text_input("Registered Email",
                                  placeholder="you@example.com", key="fp_em")
            if st.button("Check email →", key="fp_chk"):
                if not email:
                    st.error("Please enter your email.")
                else:
                    conn = sqlite3.connect("users.db")
                    c = conn.cursor()
                    c.execute("SELECT security_q FROM users WHERE email=?", (email,))
                    row = c.fetchone(); conn.close()
                    if row:
                        st.session_state.reset_email    = email
                        st.session_state.question_shown = True
                        st.success(f"Security question: **{row[0]}**")
                    else:
                        st.error("Email not found.")

            if st.session_state.question_shown:
                sp()
                answer = st.text_input("Security Answer",
                                       placeholder="Your answer", key="fp_ans")
                if st.button("Verify answer →", key="fp_ver"):
                    if " " in answer:
                        st.error("No spaces allowed.")
                    else:
                        conn = sqlite3.connect("users.db")
                        c = conn.cursor()
                        c.execute("SELECT security_a FROM users WHERE email=?",
                                  (st.session_state.reset_email,))
                        stored = c.fetchone(); conn.close()
                        if stored and verify_text(answer.lower().strip(), stored[0]):
                            st.session_state.answer_verified = True
                            st.success("Verified — set your new password below.")
                        else:
                            st.error("Incorrect answer.")

            if st.session_state.answer_verified:
                sp()
                new_pw  = st.text_input("New Password",     placeholder="Min 8 alphanumeric chars",
                                         type="password", key="fp_np")
                conf_pw = st.text_input("Confirm Password", placeholder="Same as above",
                                         type="password", key="fp_cp")
                if st.button("Reset password →", key="fp_rst"):
                    if new_pw != conf_pw:
                        st.error("Passwords do not match.")
                    elif not validate_password(new_pw):
                        st.error("Password must be alphanumeric and 8+ characters.")
                    elif " " in new_pw:
                        st.error("No spaces allowed.")
                    else:
                        conn = sqlite3.connect("users.db")
                        c = conn.cursor()
                        c.execute("UPDATE users SET password=? WHERE email=?",
                                  (hash_text(new_pw), st.session_state.reset_email))
                        conn.commit(); conn.close()
                        st.success("Password reset — please sign in.")
                        st.session_state.question_shown  = False
                        st.session_state.answer_verified = False
                        st.session_state.reset_email     = None

            sp()
            ghost_back("← Back", "home", "fp_back")

    # ── DASHBOARD ─────────────────────────────────────────────────
    elif st.session_state.page == "dashboard":
        payload = verify_token(st.session_state.get("token"))
        if payload:
            _, col, _ = st.columns([1, 1.6, 1])
            with col:
                sp("sp24")
                st.markdown(f"""
                <div class="dash-card">
                  <h2>Welcome back,<br><em>{payload['username']}</em></h2>
                  <p>{payload['email']}</p>
                </div>""", unsafe_allow_html=True)
                st.info("Your dashboard is ready. More features coming soon.")
                sp("sp16")
                ghost_back("Sign out", "home", "logout")
                if st.session_state.get("page") == "home":
                    st.session_state.clear()
                    st.session_state.page = "home"
        else:
            st.session_state.page = "login"; st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [41]:
import os
from pyngrok import ngrok

# Kill streamlit
os.system("pkill -9 streamlit")

# Kill ngrok processes
os.system("pkill -9 ngrok")

# Kill pyngrok session
ngrok.kill()

print("All previous processes killed.")

All previous processes killed.


In [42]:

import time
time.sleep(10)
print("Wait completed.")

Wait completed.


In [43]:
from pyngrok import ngrok
import subprocess
import time

NGROK_AUTH_TOKEN = "39yGsZsEsKXf5vNuCWmZ8gTZG3P_2GsMzXTvxZqUAT2uHWG7P"  # keep yours

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"]
)

time.sleep(8)

# Important: Don't reuse old domain
public_url = ngrok.connect(8501)
print("🌍 NEW URL:", public_url)

🌍 NEW URL: NgrokTunnel: "https://chubby-vania-unwonderfully.ngrok-free.dev" -> "http://localhost:8501"


In [44]:

import sqlite3
import pandas as pd

conn = sqlite3.connect('users.db')
df = pd.read_sql_query("SELECT id, username, email, security_q FROM users", conn)
conn.close()
print(df)

   id          username                        email              security_q
0   1               abc             abc123@gmail.com  What is your pet name?
1   2  abc123@gmail.com  kuruvalaramya2006@gmail.com  What is your pet name?
2   3               abC             ABC123@gmail.com  What is your pet name?
